<p style="font-size: 37.5px"><b>System Analytics By Windows Keylogger</b></p>

<p style="font-size: 14px; margin-bottom: -12px;"><i>Antonio Scardace, 1000007272</i></p>
<p style="font-size: 14px; margin-bottom: -12px;"><i>June 2022</i></p>
<p style="font-size: 14px; margin-bottom: -12px;"><i>Technologies for Advanced Programming</i></p>
<p style="font-size: 14px;"><i>Department of Mathematics and Computer Science @ University of Catania</i></p>

<p style="font-size: 35px" align="center"><b>Aim of the Project</b></p>

Make stats on the real-time use of the system by the user (and by users in general).

The following functions are available (personal and general stats):

- For Logged Text:
    + Top 8 Last Logged Texts
    + Sentiment analysis

- For Metadata:
    + Top 10 most used applications 
    + Used windows classification 
        * Social
        * Utility
        * Entertainment
        * Web Browsing
        * Office & Study
        * Other
    + Customers Geolocation by IP 
    + Overall time spent writing to the PC 
    + Average time spent writing in a window 
    + Average time spent writing in each category of window 
    + Overall time spent on each category of window

<p style="font-size: 25px" align="center"><b>Real Use Cases</b></p>

Let's take **Apple** and **Microsoft** as examples:

Usually, each company provides a personal stats interface and owns a general stats interface that monitors a high amount of data produced in real-time every second of the day by customers' computers.

It can also be used as:
1. System Monitor for **Prison Computers**
2. System Monitor for **Public Offices Computers**
3. **Parental Control**
4. **Spyware**.

Can it be done with old technologies? <br/>
Yes, it can. BUT...

<p style="font-size: 35px" align="center"><b>Data Source</b></p>

Is a Windows Keylogger for Italian keyboards only.<br/>
Developed in C++14 using Windows APIs. <br/>

Sends a log to the TCP server on two events:
- On each foreground window change
- After 1 minute of user inactivity.

The log has the following pattern:
```
[UUID] :: [Window Title] :: [Timestamp Start]
Logged Text...
[Timestamp End] :: [IP Address]
```

Each log is composed by:
- **UUID**: Identifies the PC univocally. <br/> Has the following format: xxxxxxxx-xxxx-xxxx-xxxx-xxxxxxxxxxxx.
- **Window Title**: Is the title of the window where the user has typed.
- **Timestamp Start**: Indicates when the user started typing in that window.
- **Logged Text**: Is the set of keys pressed by the user and logged by the keylogger.
- **Timestamp End**: Indicates when the user finished typing in that window.
- **IP Address**: Is the public IP address. <br/> If the PC has no connection, the default value is "Unknown".

For instance:
```
[00000000-0000-0000-0000-000000000000] :: [WhatsApp] :: [2022-01-01 12:00:00]
Hi Nicole, happy new year!!
[2022-01-01 12:00:13] :: [1.2.3.4]
```

![Alt Text](gifs/peter-long-nails.gif)

<p style="font-size: 35px" align="center"><b>Server System</b></p>

It receives logs (from multiple clients) and passes them to the pipeline illustrated below:

<img src="images/pipeline.png" height="600px"/>

Me proud once the design has been finished

![Alt Text](gifs/stewie-sunday.gif)

<p style="font-size: 35px" align="center"><b>Commonalities</b></p>

<img src="logos/docker-logo.png?v=1653175210" width="150px" />

To make the project portable and scalable, each of the following components used has been put inside a **Docker Container**. All the Docker Containers have been connected via **Docker Compose**.

<p style="font-size: 35px" align="center"><b>Server</b></p>

<img src="logos/python-logo.png?v=1653175210" width="150px" />

It is a multi-threading server that serves multiple clients. <br/>
Receives logs via TCP requests on the 8800 port.

It extracts the features from the logs and saves them into two CSV files:

_metadata.csv_ = [UUID, Window Title, Timestamp of Begin, Timestamp of End, IP Address] <br/>
_logs.csv_ = [UUID, Logged Text]

<p align="center"><img src="images/server-schema.svg"/></p>

<p style="font-size: 35px" align="center"><b>Data Ingestion</b></p>

<img src="logos/logstash-logo.png" width="165px" />

I have used it to create two different data flow: one for the metadata and one for the text logs. Logstash takes this input data from two files, metadata.csv and logs.csv - they have been shared with the server container via a Docker volume.

<p style="font-size: 35px" align="center"><b>Data Streaming</b></p>

<img src="logos/kafka-logo.png" width="165px" />

I used it to make a single cluster, which has two topics: one for logs and one for metadata. It receives two different dataflows by Logstash and stores them to be pulled by Spark. **Kafka Stream** has not been used.

<p style="font-size: 35px" align="center"><b>Data Processing</b></p>

<img src="logos/spark-logo.png" width="165px" />

How I thought I'd end up after Spark

![Alt Text](gifs/stewie-drunk.gif)

I have created two Docker Containers - one for each Kafka Topic we need to read from. Each of them, after the processing, saves the docs into the ES index _keylogger_. <br/>
In the first, Spark Streaming read data from the _logs_ topic and adds a little set of features. It is the VADER dictionary. <br/>
In the second, Spark Streaming read data from the _metadata_ topic and adds three features: the type of the window, the difference (in seconds) between the two timestamps fields, and the public IP address geolocation coordinates (if it isn't set to "Unknown").

<p style="font-size: 35px" align="center"><b>Data Storing</b></p>

<img src="logos/elasticsearch-logo.png" width="275px" />

I have used it to create a cluster, containing the *keylogger_stats* index, shared only by a single node: *es001*. It receives docs from Spark Streaming. Data is saved into a Docker Volume to make the application persistent in time.

<p style="font-size: 35px" align="center"><b>Data Visualization</b></p>

<img src="logos/kibana-logo.png" width="275px" />

I have used it to visualize some stats in real-time - I have set the dashboard auto-refresh to happen every second. The dashboard analyzes just the data that have arrived in the last 24 hours. It shows general data (of all PCs) and if the user clicks on a specific UUID, the dashboard shows data of that particular PC. 

<p align="center"><img src="snaps/kibana-0.png"/></p>

<p align="center"><img src="snaps/kibana-1.png"/></p>

<p align="center"><img src="snaps/kibana-2.png"/></p>

<p align="center"><img src="snaps/kibana-3.png"/></p>

<p align="center"><img src="snaps/kibana-4.png"/></p>

<p style="font-size: 35px" align="center"><b>Experiments</b></p>

- Spark NLP for Logged Text Sentiment Analysis and some other analytics
- Upgrade the service: from local to cloud

<p style="font-size: 35px" align="center"><b>Requisites to getting started</b></p>


* At least 12 GB of RAM.
* At least 25 GB of free space.
* Use of Linux, MacOS, or Windows WSL.
* Need to download and install [Docker](https://docs.docker.com/get-docker/) (but the use of [Docker Desktop](https://www.docker.com/products/docker-desktop/) is optional).
* The use of [Visual Studio Code](https://code.visualstudio.com/download) is strongly recommended.

That's what the **FBI** will do to me once the project will be public... 

![Alt Text](gifs/peter-fbi.gif)

<p style="font-size: 35px" align="center"><b>THANK YOU FOR YOUR ATTENTION</b></p>

<p style="font-size: 35px" align="center"><b>LET'S SEE SOMETHING!</b></p> <br/><br/>


PLEASE USE AND READ IT FOR ACADEMIC PURPOSES ONLY. <br/>
I DISCLAIM ANY LIABILITY FOR ILLEGAL USE.